In [ ]:
from google.colab import drive

drive.mount('/content/drive')

import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/My Drive/Colab Notebooks/nth-honor-339920-5256e664e9e6.json"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
  DOCKERFILE
'''
# FROM pytorch/pytorch:latest
# WORKDIR /app
# COPY requirements.txt requirements.txt
# RUN pip install -r requirements.txt
# RUN mkdir tocrop cropped models
# COPY mask_display.png tocrop/
# COPY unsupervised_blending_gan.npz models/ 
# COPY *.py .
# EXPOSE 8080
# CMD [ "python3", "main.py" ]



# Folder Structure
# app -> tocrop, cropped, models -> tocrop:mask_display.png , models:usupervised...

'\n  DOCKERFILE\n'

In [ ]:
'''
  Reference: main.py routing calls
  Summons perform_blend.py's Merger class, sends src and dst and blended-result-name
'''

# blender = Merger()

# @app.route("/blend", methods=["GET"])
# def get_result_url():
#     # source = request.args.to_dict()['image_src']
#     # destin = request.args.to_dict()["image_dst"]
#     # blended = request.args.to_dict()["blend_id"]
    
#     # result_BLOB = blender.get_res_blob(
#     #     source, destin, blended
#     # )

#     # return jsonify({"result_BLOB": result_BLOB})
#     data_dict = request.args.to_dict()["instances"]

#     # image source # image dest # blend id
#     logging.info(data_dict)



#     source = data_dict[0]["image_src"]
#     destin = data_dict[0]["image_dst"]
#     blended = data_dict[0]["blend_id"]

#     result_BLOB = blender.get_res_blob(source, destin, blended)

#     return jsonify({"result_BLOB": result_BLOB})

"\n  Reference: main.py routing calls\n  Summons perform_blend.py's Merger class, sends src and dst and blended-result-name\n"

In [ ]:
'''
  Reference: cropper.py
'''

from skimage.io import imread, imsave
import os
# tocrop = os.path.join(os.getcwd(), 'tocrop')
# cropped = os.path.join(os.getcwd(), 'cropped')
tocrop = "/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/tocrop"
cropped = "/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/cropped"
sx,sy,ex,ey = 4,5,476,494

def cropper():
    for img in os.listdir(tocrop):
        #(root,ext) = os.path.splitext(img)
        mask = imread(str(os.path.join(tocrop, img)))
        cropped_mask = mask[sx:ex, sy:ey]
        mask_name = "crop_" + str(img)
        imsave(os.path.join(cropped, mask_name), cropped_mask)

In [ ]:
!pip install chainer==6.3.0

     |████████████████████████████████| 874 kB 13.6 MB/s 
     |████████████████████████████████| 1.2 MB 64.0 MB/s 
  Created wheel for chainer: filename=chainer-6.3.0-py3-none-any.whl size=883599 sha256=263658bd0c73f22d46db13212e48efb8ce73471c69a963ec6008e9b309e71338
  Stored in directory: /root/.cache/pip/wheels/be/67/97/6fa45a07187f9cf6968adc6c57c5af94c975e374534a56fff7
Successfully built chainer
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.

In [ ]:
'''
  model.py
'''
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import cuda


def init_conv(array):
    xp = cuda.get_array_module(array)
    array[...] = xp.random.normal(loc=0.0, scale=0.02, size=array.shape)


def init_bn(array):
    xp = cuda.get_array_module(array)
    array[...] = xp.random.normal(loc=1.0, scale=0.02, size=array.shape)


class ReLU(chainer.Chain):
    def __init__(self):
        super(ReLU, self).__init__()

    def __call__(self, x):
        return F.relu(x)


class Tanh(chainer.Chain):
    def __init__(self):
        super(Tanh, self).__init__()

    def __call__(self, x):
        return F.tanh(x)


class LeakyReLU(chainer.Chain):
    def __init__(self):
        super(LeakyReLU, self).__init__()

    def __call__(self, x):
        return F.leaky_relu(x)


class DCGAN_G(chainer.ChainList):
    def __init__(self, isize, nc, ngf, conv_init=None, bn_init=None):
        cngf, tisize = ngf // 2, 4
        while tisize != isize:
            cngf = cngf * 2
            tisize = tisize * 2

        layers = []
        # input is Z, going into a convolution
        layers.append(L.Deconvolution2D(None, cngf, ksize=4, stride=1, pad=0, initialW=conv_init, nobias=True))
        layers.append(L.BatchNormalization(cngf, initial_gamma=bn_init))
        layers.append(ReLU())
        csize, cndf = 4, cngf
        while csize < isize // 2:
            layers.append(L.Deconvolution2D(None, cngf // 2, ksize=4, stride=2, pad=1, initialW=conv_init, nobias=True))
            layers.append(L.BatchNormalization(cngf // 2, initial_gamma=bn_init))
            layers.append(ReLU())
            cngf = cngf // 2
            csize = csize * 2
        layers.append(L.Deconvolution2D(None, nc, ksize=4, stride=2, pad=1, initialW=conv_init, nobias=True))
        layers.append(Tanh())

        super(DCGAN_G, self).__init__(*layers)

    def __call__(self, x):
        for i in range(len(self)):
            x = self[i](x)

        return x


class DCGAN_D(chainer.ChainList):
    def __init__(self, isize, ndf, nz=1, conv_init=None, bn_init=None):
        layers = []
        layers.append(L.Convolution2D(None, ndf, ksize=4, stride=2, pad=1, initialW=conv_init, nobias=True))
        layers.append(LeakyReLU())
        csize, cndf = isize / 2, ndf
        while csize > 4:
            in_feat = cndf
            out_feat = cndf * 2
            layers.append(L.Convolution2D(None, out_feat, ksize=4, stride=2, pad=1, initialW=conv_init, nobias=True))
            layers.append(L.BatchNormalization(out_feat, initial_gamma=bn_init))
            layers.append(LeakyReLU())

            cndf = cndf * 2
            csize = csize / 2
        # state size. K x 4 x 4
        layers.append(L.Convolution2D(None, nz, ksize=4, stride=1, pad=0, initialW=conv_init, nobias=True))

        super(DCGAN_D, self).__init__(*layers)

    def encode(self, x):
        for i in range(len(self)):
            x = self[i](x)

        return x

    def __call__(self, x):
        x = self.encode(x)
        x = F.sum(x, axis=0) / x.shape[0]
        return F.squeeze(x)


class EncoderDecoder(chainer.Chain):
    def __init__(self, nef, ngf, nc, nBottleneck, image_size=64, conv_init=None, bn_init=None):
        super(EncoderDecoder, self).__init__(
            encoder=DCGAN_D(image_size, nef, nBottleneck, conv_init, bn_init),
            bn=L.BatchNormalization(nBottleneck, initial_gamma=bn_init),
            decoder=DCGAN_G(image_size, nc, ngf, conv_init, bn_init)
        )

    def encode(self, x):
        h = self.encoder.encode(x)
        h = F.leaky_relu(self.bn(h))

        return h

    def decode(self, x):
        h = self.decoder(x)

        return h

    def __call__(self, x):
        h = self.encode(x)
        h = self.decode(h)
        return h

/usr/local/lib/python3.7/dist-packages/chainer/backends/cuda.py:143: UserWarning: cuDNN is not enabled.
Please reinstall CuPy after you install cudnn
(see https://docs-cupy.chainer.org/en/stable/install.html#install-cudnn).
  'cuDNN is not enabled.\n'


In [ ]:
'''
  gp_gan.py
'''
import math

import chainer
import chainer.functions as F
import numpy as np
from chainer import cuda, Variable
from scipy.fftpack import dct, idct
from scipy.ndimage import correlate
from scipy.optimize import minimize
from skimage.filters import gaussian, sobel_h, sobel_v, scharr_h, scharr_v, roberts_pos_diag, roberts_neg_diag, \
    prewitt_h, prewitt_v
from skimage.transform import resize

################## Gradient Operator #########################
normal_h = lambda im: correlate(im, np.asarray([[0, -1, 1]]), mode='nearest')
normal_v = lambda im: correlate(im, np.asarray([[0, -1, 1]]).T, mode='nearest')

gradient_operator = {
    'normal': (normal_h, normal_v),
    'sobel': (sobel_h, sobel_v),
    'scharr': (scharr_h, scharr_v),
    'roberts': (roberts_pos_diag, roberts_neg_diag),
    'prewitt': (prewitt_h, prewitt_v)
}


###########################################################


def preprocess(im):
    im = np.transpose(im * 2 - 1, (2, 0, 1)).astype(np.float32)
    return im


def ndarray_resize(im, image_size, order=3, dtype=None):
    im = resize(im, image_size, preserve_range=True, order=order, mode='constant')

    if dtype:
        im = im.astype(dtype)
    return im


def z_generate(z, G, copy_paste_var, nz, gpu):
    z = np.reshape(z, (nz, 1, 1)).astype(np.float32)
    z_var = Variable(chainer.dataset.concat_examples([z], gpu))

    loss = F.mean_squared_error(copy_paste_var, G(z_var))

    # Backward
    loss.backward()
    # Transfer loss & diff from GPU to CPU
    loss = cuda.to_cpu(loss.data)
    dz = np.squeeze(cuda.to_cpu(z_var.grad))

    return loss, np.asarray(dz.flatten(), dtype=np.float64)


def imfilter2d(im, filter_func):
    gradients = np.zeros_like(im)
    for i in range(im.shape[2]):
        gradients[:, :, i] = filter_func(im[:, :, i])

    return gradients


def gradient_feature(im, color_feature, gradient_kernel):
    result = np.zeros((*im.shape, 5))

    gradient_h, gradient_v = gradient_operator[gradient_kernel]

    result[:, :, :, 0] = color_feature
    result[:, :, :, 1] = imfilter2d(im, gradient_h)
    result[:, :, :, 2] = imfilter2d(im, gradient_v)
    result[:, :, :, 3] = np.roll(result[:, :, :, 1], 1, axis=1)
    result[:, :, :, 4] = np.roll(result[:, :, :, 2], 1, axis=0)

    return result.astype(im.dtype)


def fft2(K, size, dtype):
    w, h = size
    param = np.fft.fft2(K)
    param = np.real(param[0:w, 0:h])

    return param.astype(dtype)


def laplacian_param(size, dtype):
    w, h = size
    K = np.zeros((2 * w, 2 * h)).astype(dtype)

    laplacian_k = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]])
    kw, kh = laplacian_k.shape
    K[:kw, :kh] = laplacian_k

    K = np.roll(K, -(kw // 2), axis=0)
    K = np.roll(K, -(kh // 2), axis=1)

    return fft2(K, size, dtype)


def gaussian_param(size, dtype, sigma):
    w, h = size
    K = np.zeros((2 * w, 2 * h)).astype(dtype)

    K[1, 1] = 1
    K[:3, :3] = gaussian(K[:3, :3], sigma)

    K = np.roll(K, -1, axis=0)
    K = np.roll(K, -1, axis=1)

    return fft2(K, size, dtype)


def dct2(x, norm='ortho'):
    return dct(dct(x, norm=norm).T, norm=norm).T


def idct2(x, norm='ortho'):
    return idct(idct(x, norm=norm).T, norm=norm).T


def gaussian_poisson_editing(X, param_l, param_g, color_weight=1, eps=1e-12):
    Fh = (X[:, :, :, 1] + np.roll(X[:, :, :, 3], -1, axis=1)) / 2
    Fv = (X[:, :, :, 2] + np.roll(X[:, :, :, 4], -1, axis=0)) / 2
    L = np.roll(Fh, 1, axis=1) + np.roll(Fv, 1, axis=0) - Fh - Fv

    param = param_l + color_weight * param_g
    param[(param >= 0) & (param < eps)] = eps
    param[(param < 0) & (param > -eps)] = -eps

    Y = np.zeros(X.shape[:3])
    for i in range(3):
        Xdct = dct2(X[:, :, i, 0])
        Ydct = (dct2(L[:, :, i]) + color_weight * Xdct) / param
        Y[:, :, i] = idct2(Ydct)
    return Y


def run_gp_editing(src_im, dst_im, mask_im, gan_im, color_weight, sigma, gradient_kernel='normal'):
    dst_feature = gradient_feature(dst_im, gan_im, gradient_kernel)
    src_feature = gradient_feature(src_im, gan_im, gradient_kernel)
    feature = dst_feature * (1 - mask_im) + src_feature * mask_im

    size, dtype = feature.shape[:2], feature.dtype
    param_l = laplacian_param(size, dtype)
    param_g = gaussian_param(size, dtype, sigma)
    gan_im = gaussian_poisson_editing(feature, param_l, param_g, color_weight=color_weight)
    gan_im = np.clip(gan_im, 0, 1)

    return gan_im


def laplacian_pyramid(im, max_level, image_size, smooth_sigma):
    im_pyramid = [im]
    diff_pyramid = []
    for i in range(max_level - 1, -1, -1):
        smoothed = gaussian(im_pyramid[-1], smooth_sigma, multichannel=True)
        diff_pyramid.append(im_pyramid[-1] - smoothed)
        smoothed = ndarray_resize(smoothed, (image_size * 2 ** i, image_size * 2 ** i))
        im_pyramid.append(smoothed)

    im_pyramid.reverse()
    diff_pyramid.reverse()

    return im_pyramid, diff_pyramid


"""
GP-GAN: Towards Realistic High-Resolution Image Blending
    obj:  source image,      size: w x h x 3, dtype: float, value: [0, 1]
    bg :  destination image, size: w x h x 3, dtype: float, value: [0, 1]
    mask: mask image,        size: w x h,     dtype: float, value: {0, 1}
    G: Generator
    image_size: image_size for Blending GAN
    gpu: gpu id
    color_weight: beta in Gaussion-Poisson Equation
    sigma: sigma for gaussian smooth of Gaussian-Poisson Equation
    gradient_kernel: kernel type for calc gradient
    smooth_sigma: sigma for gaussian smooth of Laplacian pyramid
    supervised: supervised Blending GAN ?
    ## If supervised = False
    nz: noise vector lendth
    n_iteration: # of iterations for optimization
"""


def gp_gan(obj, bg, mask, G, image_size, gpu, color_weight=1, sigma=0.5, gradient_kernel='normal', smooth_sigma=1,
           supervised=True, nz=100, n_iteration=1000):
    w_orig, h_orig, _ = obj.shape
    ############################ Gaussian-Poisson GAN Image Editing ###########################
    # pyramid
    max_level = int(math.ceil(np.log2(max(w_orig, h_orig) / image_size)))
    obj_im_pyramid, _ = laplacian_pyramid(obj, max_level, image_size, smooth_sigma)
    bg_im_pyramid, _ = laplacian_pyramid(bg, max_level, image_size, smooth_sigma)

    # init GAN image
    mask_init = ndarray_resize(mask, (image_size, image_size), order=0)[:, :, np.newaxis]
    copy_paste_init = obj_im_pyramid[0] * mask_init + bg_im_pyramid[0] * (1 - mask_init)
    copy_paste_init_var = Variable(chainer.dataset.concat_examples([preprocess(copy_paste_init)], gpu))

    if supervised:
        gan_im_var = G(copy_paste_init_var)
    else:
        z_init = np.random.normal(size=(nz, 1, 1))
        res = minimize(z_generate, z_init, args=(G, copy_paste_init_var, nz, gpu), method='L-BFGS-B', jac=True,
                       options={'maxiter': n_iteration, 'disp': False})
        z = np.reshape(res.x, (nz, 1, 1)).astype(np.float32)
        gan_im_var = G(Variable(chainer.dataset.concat_examples([z], gpu)))
    gan_im = np.clip(np.transpose((np.squeeze(cuda.to_cpu(gan_im_var.data)) + 1) / 2, (1, 2, 0)), 0, 1).astype(
        obj.dtype)

    # Start pyramid
    for level in range(max_level + 1):
        size = obj_im_pyramid[level].shape[:2]
        mask_im = ndarray_resize(mask, size, order=0)[:, :, np.newaxis, np.newaxis]
        if level != 0:
            gan_im = ndarray_resize(gan_im, size)

        gan_im = run_gp_editing(obj_im_pyramid[level], bg_im_pyramid[level], mask_im, gan_im, color_weight, sigma,
                                gradient_kernel)

    gan_im = np.clip(gan_im * 255, 0, 255).astype(np.uint8)

    return gan_im

In [ ]:
'''
  Reference: gp_gan_runner.py --> called by perform_blend.py
  Calls gp_gan.py and model.py
'''
import argparse
import os

import chainer
from chainer import cuda, serializers
from skimage import img_as_float
from skimage.io import imread, imsave

basename = lambda path: os.path.splitext(os.path.basename(path))[0]

"""
    Note: source image, destination image and mask image have the same size.
"""


def runner(sourceIMG, destinIMG, maskIMG, blend_id, supervised=False):
    # parser = argparse.ArgumentParser(description='Gaussian-Poisson GAN for high-resolution image blending')
    # parser.add_argument('--nef', type=int, default=64, help='# of base filters in encoder')
    nef = 64
    # parser.add_argument('--ngf', type=int, default=64, help='# of base filters in decoder or G')
    ngf = 64
    # parser.add_argument('--nc', type=int, default=3, help='# of output channels in decoder or G')
    nc = 3
    # parser.add_argument('--nBottleneck', type=int, default=4000, help='# of output channels in encoder')
    nBottleneck = 4000
    # parser.add_argument('--ndf', type=int, default=64, help='# of base filters in D')
    ndf = 64
    # parser.add_argument('--image_size', type=int, default=64, help='The height / width of the input image to network')
    image_size = 64
    # parser.add_argument('--color_weight', type=float, default=1, help='Color weight')
    color_weight = 1
    # parser.add_argument('--sigma', type=float, default=0.5,
    #                     help='Sigma for gaussian smooth of Gaussian-Poisson Equation')
    sigma = 0.5
    # parser.add_argument('--gradient_kernel', type=str, default='normal', help='Kernel type for calc gradient')
    gradient_kernel = 'normal'
    # parser.add_argument('--smooth_sigma', type=float, default=1, help='Sigma for gaussian smooth of Laplacian pyramid')
    smooth_sigma = 1
    # parser.add_argument('--supervised', type=lambda x: x == 'True', default=True,
    #                     help='Use unsupervised Blending GAN if False')
    supervised = True
    # parser.add_argument('--nz', type=int, default=100, help='Size of the latent z vector')
    nz = 100
    # parser.add_argument('--n_iteration', type=int, default=1000, help='# of iterations for optimizing z')
    n_iteration = 10000
    # parser.add_argument('--gpu', type=int, default=0, help='GPU ID (negative value indicates CPU)')
    gpu = 0
    # parser.add_argument('--g_path', default='models/blending_gan.npz', help='Path for pretrained Blending GAN model')
    g_path = '/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/models/blending_gan.npz'
    # parser.add_argument('--unsupervised_path', default='models/unsupervised_blending_gan.npz',
    #                     help='Path for pretrained unsupervised Blending GAN model')
    unsupervised_path='/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/models/unsupervised_blending_gan.npz'
    # parser.add_argument('--list_path', default='',
    #                     help='File for input list in csv format: obj_path;bg_path;mask_path in each line')
    list_path =''
    # parser.add_argument('--result_folder', default='blending_result', help='Name for folder storing results')
    result_folder = 'blending_result'
    # parser.add_argument('--src_image', default='', help='Path for source image')
    src_image = sourceIMG
    # parser.add_argument('--dst_image', default='', help='Path for destination image')
    dst_image = destinIMG
    # parser.add_argument('--mask_image', default='', help='Path for mask image')
    mask_image = maskIMG
    # parser.add_argument('--blended_image', default='', help='Where to save blended image')
    blended_image = blend_id
    # args = parser.parse_args()

    # print('Input arguments:')
    # for key, value in vars(args).items():
    #     print('\t{}: {}'.format(key, value))
    # print('')

    # Init CNN model
    if supervised:
        G = EncoderDecoder(nef, ngf, nc, nBottleneck, image_size=image_size)
        print('Load pretrained Blending GAN model from {} ...'.format(g_path))
        serializers.load_npz(g_path, G)
    else:
        chainer.config.use_cudnn = 'never'
        G = DCGAN_G(image_size, nc, ngf)
        print('Load pretrained unsupervised Blending GAN model from {} ...'.format(unsupervised_path))
        serializers.load_npz(unsupervised_path, G)

    if gpu >= 0:
        cuda.get_device(gpu).use()  # Make a specified GPU current
        G.to_gpu()  # Copy the model to the GPU

    # Init image list
    if list_path:
        print('Load images from {} ...'.format(list_path))
        with open(list_path) as f:
            test_list = [line.strip().split(';') for line in f]
        print('\t {} images in total ...\n'.format(len(test_list)))
    else:
        test_list = [(src_image, dst_image, mask_image)]

    if not blended_image:
        # Init result folder
        if not os.path.isdir(result_folder):
            os.makedirs(result_folder)
        print('Result will save to {} ...\n'.format(result_folder))

    total_size = len(test_list)
    for idx in range(total_size):
        print('Processing {}/{} ...'.format(idx + 1, total_size))

        # load image
        obj = img_as_float(imread(test_list[idx][0]))
        bg = img_as_float(imread(test_list[idx][1]))
        mask = imread(test_list[idx][2], as_gray=True).astype(obj.dtype)

        with chainer.using_config("train", False):
            blended_im = gp_gan(obj, bg, mask, G, image_size, gpu, color_weight=color_weight,
                                sigma=sigma,
                                gradient_kernel=gradient_kernel, smooth_sigma=smooth_sigma,
                                supervised=supervised,
                                nz=nz, n_iteration=n_iteration)

        if blended_image:
            imsave(blended_image, blended_im)
        else:
            imsave('{}/obj_{}_bg_{}_mask_{}.png'.format(result_folder, basename(test_list[idx][0]),
                                                        basename(test_list[idx][1]), basename(test_list[idx][2])),
                   blended_im)

In [ ]:
'''
  reference: perform_blend.py
  Class Definition of Merger
  main.py calls get_res_blob(self, src1_prefix, src2_prefix, blend_blob_name)
'''

import os
from google.cloud import storage
import datetime


class Merger:
    
    def list_buckets():
        """Lists all buckets."""

        storage_client = storage.Client()
        buckets = storage_client.list_buckets()

        for bucket in buckets:
            print(bucket.name)
    
    def generate_download_signed_url_v4(bucket_name, blob_name):
        """Generates a v4 signed URL for downloading a blob.
        Note that this method requires a service account key file. You can not use
        this if you are using Application Default Credentials from Google Compute
        Engine or from the Google Cloud SDK.
        """
        # bucket_name = 'your-bucket-name'
        # blob_name = 'your-object-name'

        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        #print(bucket)
        blob = bucket.blob(blob_name)
        #print(bucket.list_blobs())
        # for sub_folder in bucket.list_blobs():
        #   print(sub_folder)
        url = blob.generate_signed_url(
            version="v4",
            # This URL is valid for 15 minutes
            expiration=datetime.timedelta(minutes=15),
            # Allow GET requests using this URL.
            method="GET",
        )
        # print("You can use this URL with any user agent, for example:")
        # print("curl '{}'".format(url))
        return url
    
    def upload_blob(bucket_name, source_file_name, destination_blob_name):
        """Uploads a file to the bucket."""
        # The ID of your GCS bucket
        # bucket_name = "your-bucket-name"
        # The path to your file to upload
        # source_file_name = "local/path/to/file"
        # The ID of your GCS object
        # destination_blob_name = "storage-object-name"

        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(destination_blob_name)

        blob.upload_from_filename(source_file_name)

        print(
            "File {} uploaded to {}.".format(
                source_file_name, destination_blob_name
            )
        )

    def download_blob(bucket_name, source_blob_name, destination_file_name):
        """Downloads a blob from the bucket."""
        storage_client = storage.Client()
        try:
            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(source_blob_name)
            blob.download_to_filename(destination_file_name)
            print('file: ',destination_file_name,' downloaded from bucket: ',bucket_name,' successfully')
        except Exception as e:
            print(e)

    def get(self):
        bucket_name = os.environ.get("imagesearch-cs329s",
                                    app_identity.get_default_gcs_bucket_name())

        self.response.headers['Content-Type'] = 'text/plain'
        self.response.write('Demo GCS Application running from Version: '
                            + os.environ['CURRENT_VERSION_ID'] + '\n')
        self.response.write('Using bucket name: ' + bucket_name + '\n\n')

    # def get_res_url(self, source_url, destin_url, blend_id):
    #     src_data = requests.get(source_url).content
    #     destin_data = requests.get(destin_url).content
       
    #     # with open(os.path.join(os.getcwd(),'tocrop', 'src.jpg'), 'wb') as handler:
    #     #     handler.write(src_data)
    #     #     #handler.close()

    #     #os.replace('src.jpg', os.path.join(os.getcwd(),'tocrop', 'src.jpg'))
       
        

    #     # with open(os.path.join(os.getcwd(),'tocrop', 'dst.jpg'), 'wb') as handler:
    #     #     handler.write(destin_data)
    #     #     #handler.close()
    #     #os.replace('dst.jpg', os.path.join(os.getcwd(),'tocrop', 'dst.jpg'))
    #     # mask.jpg is local
    #     mask = os.path.join(os.getcwd(),'tocrop', 'mask_display.png')
        
    #     cropper(source_url,destin_url)
    #     # os.mkdir(os.path.join(os.getcwd(), 'tocrop'))
    #     # os.mkdir(os.path.join(os.getcwd(), 'cropped'))
    #     sourceIMG = os.path.join(os.getcwd(), 'cropped', 'crop_src.jpg')
    #     destinIMG = os.path.join(os.getcwd(), 'cropped', 'crop_dst.jpg')
    #     maskIMG = os.path.join(os.getcwd(), 'cropped', 'crop_mask.jpg')

    #     runner(sourceIMG, destinIMG, maskIMG, blend_id, False)
    #     blend_id_name = blend_id + '.jpg'
    #     gan_result = os.path.join(os.getcwd(), blend_id_name)

    #     upload_blob("imagesearch-cs329s", gan_result , "gan_result.jpg")

    #     return generate_download_signed_url_v4("imagesearch-cs329s","gan_result.jpg")

    def get_first_blob(self, bucket_name, prefix, delimiter = None):
        bucket_name = "imagesearch-cs329s"
        storage_client = storage.Client()
        blobs = storage_client.list_blobs(bucket_name, prefix = prefix)

        if blobs:

            count = 0
            for blob in blobs:
                if count == 1:
                    filename = os.path.join('tocrop',str(blob.name).split('/')[1])
                    blob.download_to_filename(filename)
                    return filename
                else:
                    count = count + 1

    def get_res_blob(self, src1_prefix, src2_prefix, blend_blob_name):
        # src1_img_name = str(self.get_first_blob("imagesearch-cs329s", src1_prefix).split('/')[1])
        # src2_img_name = str(self.get_first_blob("imagesearch-cs329s", src2_prefix).split('/')[1])
        # src1_img = self.get_first_blob("imagesearch-cs329s", src1_prefix)
        # src2_img = self.get_first_blob("imagesearch-cs329s", src2_prefix)
        src1_img_name = src1_prefix + '.jpg'
        src2_img_name = src2_prefix + '.jpg'

        #os.replace('mask_display.png', os.path.join(os.getcwd(),'tocrop', 'mask.png'))
        mask_name = 'mask_middle.png'
        
        cropper()
        sourceIMG = os.path.join(cropped, 'crop_'+ src1_img_name )
        destinIMG = os.path.join(cropped, 'crop_' + src2_img_name)
        maskIMG = os.path.join(cropped, 'crop_' + mask_name)
        runner(sourceIMG, destinIMG, maskIMG, blend_blob_name+'.jpg', False)
        blend_id_name = blend_blob_name + '.jpg'
        gan_result = os.path.join(os.getcwd(), blend_id_name)

        #upload_blob("imagesearch-cs329s", gan_result , "blended_gan_result.jpg")

        #return generate_download_signed_url_v4("imagesearch-cs329s","gan_result.jpg")
        return "blended_gan_result.jpg"

In [ ]:
import os
chdir = "/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output"
os.chdir(chdir)
!pip install -r requirements.txt

     |████████████████████████████████| 3.1 MB 12.8 MB/s 
     |████████████████████████████████| 26.3 MB 1.2 MB/s 
  ERROR: Failed building wheel for cupy
  Running setup.py clean for cupy
Failed to build cupy
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:
      Successfully uninstalled scikit-image-0.18.3
    Running setup.py install for cupy ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-7isuexoq/cupy_6c1872b3f893470ba5374ab71406897c/setup.py'"'"'; __file__='"'"'/tmp/pip-install-7isuexoq/cupy_6c1872b3f893470ba5374ab71406897c/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' 

In [ ]:
blender = Merger()

source = 'src_nature'
destin = 'dst_nature'
blended = 'blended_gan_result2'

result_BLOB = blender.get_res_blob(source, destin, blended)

Load pretrained unsupervised Blending GAN model from /content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/models/unsupervised_blending_gan.npz ...
Processing 1/1 ...


In [ ]:
# 4,5,476,494

# crop high-res living room


# Importing Image class from PIL module
from PIL import Image
 
# Opens a image in RGB mode
im = Image.open(r"/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/tocrop/blue-arm-chair.jpg")
os.chdir("/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/cropped")

# Size of the image in pixels (size of original image)
# (This is not mandatory)
width, height = im.size
'''4,5,476,494'''
 
# Setting the points for cropped image
left = 1*width/7
top = height / 5
right = 6*width/7
bottom = 4*height/5
 
# Cropped image of above dimension
# (It will not change original image)
#im1 = im.crop((left, top, right, bottom))
newsize = (472, 489)
im1 = im.resize(newsize)

im1.save("scaledblue-arm-chair.png")

In [ ]:
# from PIL import image

# immask = Image.open(r"/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/tocrop/scaledantique-coffee-table.jpg")
# mask_img = Image.open(immask)
# datas_mask = mask_img.getdata()

# newData = []
# for item in range(len(datas_mask)):
#     if datas_fore[item][0] == 255 and datas_fore[item][1] == 255 and datas_fore[item][2] == 255:
#         newData.append((datas_back[item][0], datas_back[item][1], datas_back[item][2]))
#     else:
#         newData.append((datas_fore[item][0], datas_fore[item][1], datas_fore[item][2]))
#     print(datas_fore[item][0])


# mask_img.putdata(newData)
# mask_img.save("mask_coffee.png", "PNG")

In [ ]:
# generate white mask
from PIL import Image

immask = Image.open(r"/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/tocrop/scaledantique-coffee-table.jpg")
os.chdir("/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/tocrop")

datas_mask = immask.getdata()

newData = []
for item in range(len(datas_mask)):
    newData.append((255,255,255))

immask.putdata(newData)
immask.save("mask_coffee.png", "PNG")


In [ ]:
# padding coffee table
from PIL import Image, ImageOps

im_pth_coffee = "/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/tocrop/scaledantique-coffee-table.jpg"
im_pth_blue = "/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/cropped/scaledblue-arm-chair.png"

os.chdir("/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/cropped")

imc = Image.open(im_pth_coffee)
imb = Image.open(im_pth_blue)
#old_size = im.size  # old_size[0] is in (width, height) format

# ratio = float(desired_size)/max(old_size)
# new_size = tuple([int(x*ratio) for x in old_size])
# use thumbnail() or resize() method to resize the input image

# thumbnail is a in-place operation

# im.thumbnail(new_size, Image.ANTIALIAS)

imc = imc.resize((170,170))
imb = imb.resize((230,240))
# create a new image and paste the resized on it

new_im = Image.new("RGB", (472,489), (255,255,255)) #white background
new_im.paste(imc, (0,280))
new_im.paste(imb, (165,270))

new_im.save('both_objects_white.jpg')

In [ ]:
def demo_runner(destinIMG_test):
  from PIL import Image

  os.chdir("/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/cropped")

  foreground = "/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/cropped/both_objects_white.jpg"
  background = destinIMG_test
  fore_img = Image.open(foreground)
  back_img = Image.open(background)
  #fore_img = fore_img.convert("RGBA")
  datas_fore = fore_img.getdata()
  datas_back = back_img.getdata()

  newData = []
  for item in range(len(datas_fore)):
      if datas_fore[item][0] + datas_fore[item][1] + datas_fore[item][2] > 700:
          newData.append((datas_back[item][0], datas_back[item][1], datas_back[item][2]))
      else:
          newData.append((datas_fore[item][0], datas_fore[item][1], datas_fore[item][2]))
      #print(datas_fore[item][0])


  fore_img.putdata(newData)
  fore_img.save("gan_result_maskwhite.jpg")

In [ ]:
os.chdir("/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images")
sourceIMG_test = "/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/cropped/gan_result_maskwhite.jpg"
destinIMG_test = "/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/tocrop/scaledbigliving.png"
maskIMG_test = "/content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/images/left_right_images/tocrop/mask_coffee.png"

demo_runner(destinIMG_test)

blend_blob_name = "demo_gan"


# Warning: using unsupervised (red-tint) model
runner(sourceIMG_test, destinIMG_test, maskIMG_test, blend_blob_name+'.jpg', False)

Load pretrained Blending GAN model from /content/drive/My Drive/Final Project CS 329S/Blending GAN/Blending Sandbox Output/models/blending_gan.npz ...
Processing 1/1 ...
